<div style="background-color:#000;"><img src="pqn.png"></img></div>

This code uses the OpenBB Terminal SDK to load historical forex data and apply signal processing techniques to detect trading signals. It computes the Hilbert Transform to derive the dominant cycle phase and period, applies a Butterworth bandpass filter, and calculates amplitude and exponential moving average of amplitude. With these transformations, it identifies buy and sell positions based on signal thresholds and plots the results. This approach is useful for analyzing market cycles and testing trading strategies.

In [ ]:
from math import pi

In [ ]:
from scipy.signal import butter, filtfilt
import numpy as np
import talib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from openbb_terminal.sdk import openbb

Load historical forex data for the EUR/USD pair between the specified dates

In [ ]:
data = openbb.forex.load(
    from_symbol="EUR", 
    to_symbol="USD", 
    start_date="2016-01-01", 
    end_date="2021-12-31"
)

Load and preprocess forex data, converting adjusted close prices to a DataFrame

In [ ]:
data = openbb.forex.load(
    from_symbol="EUR", 
    to_symbol="USD", 
    start_date="2016-01-01", 
    end_date="2021-12-31"
)

In [ ]:
prices = (
    data["Adj Close"]
    .to_frame()
    .rename(
        columns={
            "Adj Close": "close"
        }
    )
)

Calculate log returns from the closing prices

In [ ]:
prices["log_return"] = (
    prices.close
    .apply(np.log)
    .diff(1)
)

Compute the Hilbert Transform - Dominant Cycle Phase

In [ ]:
prices["phase"] = talib.HT_DCPHASE(prices.close)

Convert the phase into a sinusoidal signal

In [ ]:
prices["signal"] = np.sin(prices.phase + pi / 4)

Compute the Hilbert Transform - Dominant Cycle Period

In [ ]:
prices["period"] = talib.HT_DCPERIOD(prices.close)

In [ ]:
def butter_bandpass(data, period, delta=0.5, fs=5):
    """Applies Butterworth bandpass filter to data
    
    Parameters
    ----------
    data : np.ndarray
        The data to be filtered
    period : float
        Dominant cycle period
    delta : float, optional
        Delta value to adjust cutoff frequencies, by default 0.5
    fs : int, optional
        Sampling frequency, by default 5
    
    Returns
    -------
    np.ndarray
        Filtered data
    
    Notes
    -----
    This function applies a bandpass filter to the data using 
    the specified period and delta to determine cutoff frequencies.
    """
    
    nyq = 0.5 * fs

    # Low cutoff frequency
    low = 1.0 / (period * (1 + delta))
    low /= nyq

    # High cutoff frequency
    high = 1.0 / (period * (1 - delta))
    high /= nyq

    b, a = butter(2, [low, high], btype="band")

    return filtfilt(b, a, data)

In [ ]:
def roll_apply(e):
    """Applies Butterworth bandpass filter to rolling window data
    
    Parameters
    ----------
    e : pd.Series
        Rolling window of data
    
    Returns
    -------
    float
        Filtered value for the last data point in the window
    
    Notes
    -----
    This function takes a rolling window of data, applies the 
    Butterworth bandpass filter, and returns the filtered value 
    for the last data point in the window.
    """
    
    close = prices.close.loc[e.index]
    period = prices.period.loc[e.index][-1]
    out = butter_bandpass(close, period)
    return out[-1]

Apply rolling window to compute filtered signal using Butterworth bandpass filter

In [ ]:
prices["filtered"] = (
    prices.dropna()
    .rolling(window=30)
    .apply(lambda series: roll_apply(series), raw=False)
    .iloc[:, 0]
)

Calculate amplitude from the filtered signal over a rolling window

In [ ]:
prices["amplitude"] = (
    prices.
    filtered
    .rolling(window=30)
    .apply(
        lambda series: series.max() - series.min()
    )
)

Compute exponential moving average of amplitude

In [ ]:
prices["ema_amplitude"] = (
    talib
    .EMA(
        prices.amplitude,
        timeperiod=30
    )
)

Define signal and amplitude thresholds for position determination

In [ ]:
signal_thresh = 0.75
amp_thresh = 0.004  # 40 pips

In [ ]:
prices["position"] = 0

Determine short positions based on signal and amplitude thresholds

In [ ]:
prices.loc[
    (prices.signal >= signal_thresh) & 
    (prices.amplitude > amp_thresh), "position"
] = -1

Determine long positions based on signal and amplitude thresholds

In [ ]:
prices.loc[
    (prices.signal <= -signal_thresh) & 
    (prices.amplitude > amp_thresh), "position"
] = 1

Plot the amplitude, signal, and position time series

In [ ]:
fig, axes = plt.subplots(
    nrows=3,
    figsize=(15, 10),
    sharex=True
)

In [ ]:
prices.ema_amplitude.plot(
    ax=axes[0],
    title="amp"
)
axes[0].axhline(
    amp_thresh,
    lw=1,
    c="r"
)
prices.signal.plot(
    ax=axes[1],
    title="signal"
)
axes[1].axhline(
    signal_thresh,
    c="r"
)
axes[1].axhline(
    -signal_thresh,
    c="r"
)
prices.position.plot(
    ax=axes[2],
    title="position"
)
fig.tight_layout()

Calculate strategy returns and cumulative returns based on positions

In [ ]:
prices["strategy_return"] = prices.position.shift(1) * prices.log_return
prices["strategy_cum_return"] = prices.strategy_return.cumsum().apply(np.exp)
prices["bh_rtn_cum"] = prices["log_return"].cumsum().apply(np.exp)

Plot cumulative returns of the buy-and-hold strategy and the trading strategy

In [ ]:
(
    prices[["bh_rtn_cum", "strategy_cum_return"]]
    .plot(title="Cumulative returns")
)

Create a copy of the prices DataFrame for further analysis

In [ ]:
df = prices.copy()

Identify local minima and maxima in the signal for visualization

In [ ]:
n = 5  # number of points to be checked before and after

In [ ]:
df["min"] = df.iloc[argrelextrema(df.signal.values, np.less_equal, order=n)[0]]["signal"]
df["max"] = df.iloc[argrelextrema(df.signal.values, np.greater_equal, order=n)[0]][
    "signal"
]

Plot the identified local minima and maxima along with the signal

In [ ]:
plt.scatter(df.index, df["min"], c="r")
plt.scatter(df.index, df["max"], c="g")
plt.plot(df.index, df["signal"])
plt.show()

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.